In [1]:
import rasterio as rio
import pandas as pd
from pathlib import Path
import numpy as np
import sys

sys.path.append(Path.cwd().parents[1].as_posix())
import omnicloudmask

In [2]:
ocm_version = omnicloudmask.__version__
print(f"OmniCloudMask version: {ocm_version}")

OmniCloudMask version: 1.5.0


In [3]:
dataset_dir = Path("dataset")
val_points_path = dataset_dir / "PixBox-S2-CMIX/pixbox_sentinel2_cmix_20180425.csv"

In [4]:
# Point at the predictions directory
preds_dir = dataset_dir / f"OCM preds v{ocm_version}"
print(f"Predictions directory: {preds_dir}")
preds_dir.exists()

Predictions directory: dataset/OCM preds v1.5.0


True

In [5]:
# Load the validation data
val_data = pd.read_csv(val_points_path)
val_data.head()

,ID,PRODUCT_ID,PIXEL_X,PIXEL_Y,LATITUDE,LONGITUDE,SURFACE_ID,CLOUD_CHARACTERISTICS_ID,CLOUD_TYPE_ID,CLOUD HEIGHT_ID,...,AEROSOL TYPE_ID,GLINT_ID,WATER BODY CHARACTERISTICS_ID,ICE TYPE_ID,OVERSATURATION_ID,SURFACE_TYPE_ID,CLIMATE_ZONE_ID,SEASON_ID,DAY_TIME_ID,SHALLOWNESS_ID
0,5472574,5472572,8113,4879,-19.431860,-68.227310,27,6,1,1,...,1,1,1,1,1,6,4,1,0,1
1,5472575,5472572,8693,4333,-19.382278,-68.172318,27,6,1,1,...,1,1,1,1,1,6,4,1,0,1
2,5472576,5472572,8997,3485,-19.305517,-68.143768,27,6,1,1,...,1,1,1,1,1,6,4,1,0,1
3,5472577,5472572,9360,2757,-19.239567,-68.109573,27,6,1,1,...,1,1,1,1,1,6,4,1,0,1
4,5472578,5472572,8148,2454,-19.212711,-68.225014,27,6,1,1,...,1,1,1,1,1,6,4,1,0,1


In [6]:
# Add mapping from product_id to scene_name

product_id_to_scene_name = {
    5472572: "S2A_MSIL1C_20171205T143751_N0206_R096_T19KEU_20171205T180316",
    183807605: "S2A_MSIL1C_20170929T211511_N0205_R143_T06VWN_20170929T211510",
    273136892: "S2A_MSIL1C_20180126T182631_N0206_R127_T11SPC_20180126T201415",
    433871481: "S2A_MSIL1C_20180303T140051_N0206_R067_T19FDV_20180303T202004",
    522655885: "S2A_MSIL1C_20171107T150421_N0206_R125_T22WES_20171107T165127",
    623596858: "S2A_MSIL1C_20170629T103021_N0205_R108_T31TFJ_20170629T103020",
    701404038: "S2A_MSIL1C_20170908T100031_N0205_R122_T32SPJ_20170908T100655",
    784585929: "S2B_MSIL1C_20170731T102019_N0205_R065_T33VVE_20170731T102348",
    872013732: "S2A_MSIL1C_20180223T092031_N0206_R093_T36VUM_20180223T113049",
    885047116: "S2B_MSIL1C_20171210T060229_N0206_R091_T42RUN_20171210T071154",
    960993216: "S2A_MSIL1C_20180106T032121_N0206_R118_T48NUG_20180106T083912",
    1041598282: "S2A_MSIL1C_20180222T012651_N0206_R074_T54TWN_20180222T031349",
    1126775487: "S2B_MSIL1C_20170725T183309_N0205_R127_T11SPC_20170725T183309",
    1211409580: "S2A_MSIL1C_20170209T154541_N0204_R111_T17PPK_20170209T154543",
    1248336059: "S2A_MSIL1C_20170725T142751_N0205_R053_T19GBQ_20170725T143854",
    1309065466: "S2B_MSIL1C_20170712T113319_N0205_R080_T28PCV_20170712T114542",
    1386821964: "S2A_MSIL1C_20170726T102021_N0205_R065_T33VVE_20170726T102259",
    1407300752: "S2A_MSIL1C_20170113T072241_N0204_R006_T40UEE_20170113T072238",
    1470797653: "S2A_MSIL1C_20180217T053911_N0206_R005_T44SKJ_20180217T082149",
    1559102360: "S2A_MSIL1C_20170917T052641_N0205_R105_T48XWG_20170917T052642",
    1650478641: "S2A_MSIL1C_20170916T143741_N0205_R096_T19KEU_20170916T143942",
    1727138395: "S2A_MSIL1C_20170620T181921_N0205_R127_T11SPC_20170620T182846",
    1727140056: "S2A_MSIL1C_20180302T142851_N0206_R053_T19GBQ_20180302T192732",
    1821333386: "S2B_MSIL1C_20180302T150259_N0206_R125_T22WES_20180302T183800",
    1832501017: "S2B_MSIL1C_20170728T101029_N0205_R022_T32TPS_20170728T101024",
    1892503998: "S2B_MSIL1C_20170916T101019_N0205_R022_T32SPJ_20170916T101354",
    1899752240: "S2A_MSIL1C_20170712T071621_N0205_R006_T40UEE_20170712T071617",
    2019111565: "S2A_MSIL1C_20170706T051651_N0205_R062_T48XWG_20170706T051649",
    2065997836: "S2A_MSIL1C_20180102T140051_N0206_R067_T21LXK_20180102T154324",
}
val_data["SCENE_NAME"] = val_data["PRODUCT_ID"].map(product_id_to_scene_name)

In [7]:
# Reclassify the validation data
val_data["CLOUD"] = (
    val_data["CLOUD_CHARACTERISTICS_ID"]
    .isin([2, 3, 4, 5, 6, 8, 9, 10, 11, 12])
    .astype(bool)
)
val_data["NOT_CLOUD"] = (
    val_data["CLOUD_CHARACTERISTICS_ID"].isin([0, 1, 7]).astype(bool)
)

val_data["NOT_SHADOW"] = val_data["SHADOW_ID"].isin([0, 1, 2, 4]).astype(bool)

val_data["SHADOW"] = (val_data["SHADOW_ID"] == 3).astype(bool)

# if the pixel is not cloud or shadow, it is clear
val_data["CLEAR"] = (val_data["NOT_CLOUD"] & val_data["NOT_SHADOW"]).astype(bool)

In [8]:
print(f"Clouds: {val_data['CLOUD'].sum()}")
print(f"Shadows: {val_data['SHADOW'].sum()}")
print(f"Clear: {val_data['CLEAR'].sum()}")

Clouds: 8169
Shadows: 1246
Clear: 8297


In [9]:
# # drop rows that are nor cloud, shadow or clear
val_data = val_data[val_data["CLOUD"] | val_data["SHADOW"] | val_data["CLEAR"]]
val_data.shape

(17351, 27)

In [10]:
val_data.shape

(17351, 27)

In [11]:
# Loop over scenes and add OCM predictions
scene_results = []
for PRODUCT_ID in product_id_to_scene_name.keys():
    val_df_filt = val_data[val_data["PRODUCT_ID"] == PRODUCT_ID].copy()
    scene_name = product_id_to_scene_name[PRODUCT_ID]
    ocm_pred_path = list(preds_dir.glob(f"{scene_name}_OCM_*.tif"))[0]
    pred_array = rio.open(ocm_pred_path).read(1)
    val_df_filt["OCM_PRED"] = pred_array[
        val_df_filt["PIXEL_Y"].values, val_df_filt["PIXEL_X"].values
    ]
    scene_results.append(val_df_filt)

In [12]:
# Combine the results back into a single dataframe
scene_results_df = pd.concat(scene_results)

In [13]:
# Reclassify the OCM predictions combining thick and thin clouds
scene_results_df["OCM_CLOUD"] = scene_results_df["OCM_PRED"].isin([1, 2]).astype(bool)
scene_results_df["OCM_SHADOW"] = scene_results_df["OCM_PRED"].isin([3]).astype(bool)
scene_results_df["OCM_CLEAR"] = scene_results_df["OCM_PRED"].isin([0]).astype(bool)

In [14]:
print(scene_results_df["OCM_CLEAR"].value_counts())
print(scene_results_df["OCM_CLOUD"].value_counts())
print(scene_results_df["OCM_SHADOW"].value_counts())

OCM_CLEAR
True     8682
False    8669
Name: count, dtype: int64
OCM_CLOUD
False    9718
True     7633
Name: count, dtype: int64
OCM_SHADOW
False    16315
True      1036
Name: count, dtype: int64


In [15]:
# Get the stats for the predictions
def get_stats(labels, preds):
    tp = np.sum(labels * preds)
    tn = np.sum((1 - labels) * (1 - preds))
    fp = np.sum((1 - labels) * preds)
    fn = np.sum(labels * (1 - preds))
    ua = tp / (tp + fp)
    pa = tp / (tp + fn)
    return {
        "TP": tp,
        "TN": tn,
        "FP": fp,
        "FN": fn,
        "UA": ua,
        "PA": pa,
        "OA": (tp + tn) / (tp + tn + fp + fn),
        "BOA": 0.5 * (pa + (tn / (tn + fp))),
    }

In [16]:
#  Get the stats for each class
class_stats = {}
for class_name in ["CLEAR", "CLOUD", "SHADOW"]:
    labels = scene_results_df[class_name]
    preds = scene_results_df[f"OCM_{class_name}"]
    stats = get_stats(labels, preds)
    class_stats[f"{class_name}"] = stats

In [17]:
model_summary = pd.DataFrame(class_stats).T

In [18]:
# Format the model summary
for col in ["UA", "PA", "OA", "BOA"]:
    model_summary[col] = model_summary[col].map(lambda x: "{:.2%}".format(x))
for col in ["TP", "TN", "FP", "FN"]:
    model_summary[col] = model_summary[col].map(lambda x: "{:.0f}".format(x))

In [19]:
model_summary

,TP,TN,FP,FN,UA,PA,OA,BOA
CLEAR,7818,8190,864,479,90.05%,94.23%,92.26%,92.34%
CLOUD,7164,8713,469,1005,93.86%,87.70%,91.50%,91.29%
SHADOW,792,15861,244,454,76.45%,63.56%,95.98%,81.02%
